In [1]:
#################################3
# new model builder

import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
import albumentations as A
import os
import random
from tensorflow.keras.callbacks import ModelCheckpoint


train_dir = 'O:/project/dataset/train'
val_dir = 'O:/project/dataset/val'
checkpoint_dir = 'O:/project/epochs/mobile_adam_efined50_newmodel_freezed'
initial_learning_rate = 1e-4
epochs = 300

# Parameters
batch_size = 64
img_height = 160
img_width = 160
num_classes = 24 
val_dir = 'O:/project/dataset/val'
adv_val_dir = 'O:/project/dataset/fsgm_mobile_adam_fined_e50/epsilon_0.1/val'
model_path = 'O:/project/epochs/check_mobilenetv2_adam_4_e50_fined_50/model_epoch_50.keras'

def preprocess(image, label):
    image = tf.cast(image, tf.float32) / 255.0 
    return image, label


test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    val_dir,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    label_mode='categorical' 
).map(preprocess)

adv_test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    adv_val_dir,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    label_mode='categorical'
).map(preprocess)

model = tf.keras.models.load_model(model_path)

# 3. Remove the last layers (classifier)
layer_name = model.layers[-2].name  # Get the name of the layer before the final classifier layer
model_without_classifier = models.Model(inputs=model.input, outputs=model.get_layer(layer_name).output)

# 4. Extract Features for Test and Adversarial Test datasets
def extract_features(model, dataset):
    features = []
    labels = []
    for images, lbls in dataset:
        feats = model.predict(images)
        features.append(feats)
        labels.append(lbls)
    return np.concatenate(features), np.concatenate(labels)

test_features, test_labels = extract_features(model_without_classifier, test_dataset)
adv_test_features, _ = extract_features(model_without_classifier, adv_test_dataset)

# 5. Compare the Outputs (identify significant node changes)
node_changes = np.abs(test_features - adv_test_features).mean(axis=0)

# Determine a threshold to ignore nodes (you can define it based on your criteria)
threshold = np.mean(node_changes) + np.std(node_changes)  # Example threshold
ignored_nodes = node_changes > threshold

print(f"Threshold: {threshold}")
a = np.sum(ignored_nodes)
print(f"Number of ignored nodes: {a}")

# 6. Save ignored_nodes, threshold, and a to a text file
with open('ignored_nodes_info.txt', 'w') as file:
    file.write(f"choosen: {model_path}\n")
    file.write(f"Threshold: {threshold}\n")
    file.write(f"Number of ignored nodes: {a}\n")
    file.write("Ignored nodes:\n")
    file.write(np.array2string(ignored_nodes, separator=', '))









def albumentations_preprocessing(image):
    aug_list = [
        A.GaussianBlur(p=0.0),  # original image
        A.GaussianBlur(p=0.0),  # original image
        A.GaussianBlur(p=1.0),
        A.CoarseDropout(max_holes=2, max_height=16, max_width=16, p=1.0),
        A.GaussNoise(var_limit=(10.0, 50.0), p=1.0),
        A.RandomBrightnessContrast(brightness_limit=0.1, contrast_limit=0.1, p=1.0),
        A.ChannelShuffle(p=1.0)
    ]

    augmentation = random.choice(aug_list)
    transform = A.Compose([augmentation])
    augmented = transform(image=np.array(image))
    return augmented['image']

train_datagen = ImageDataGenerator(
    rescale=1./255,
    preprocessing_function=albumentations_preprocessing
)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

val_datagen = ImageDataGenerator(rescale=1./255)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

from tensorflow.keras.layers import Dense, Lambda

def apply_mask(x, mask):
    return x * tf.cast(tf.expand_dims(tf.constant(mask), 0), x.dtype)
for layer in model_without_classifier.layers:
    layer.trainable = False
# Apply the mask using a Lambda layer
masked_output = Lambda(lambda x: apply_mask(x, ~ignored_nodes))(model_without_classifier.output)

# Define the number of classes for your new Dense layer
num_classes = train_generator.num_classes

# Add the Dense layer to the masked output
final_output = Dense(num_classes, activation='softmax', name='classifier_dense')(masked_output)

# Create the new model
model_with_masked_output = models.Model(inputs=model_without_classifier.input, outputs=final_output)

# model.summary()
# 5. Compile and train the new model as before
model_with_masked_output.compile(
    optimizer=Adam(learning_rate=initial_learning_rate),
    loss='categorical_crossentropy',
    metrics=['accuracy', 'categorical_accuracy', 'Precision', 'Recall']
)

# 6. Train the model
os.makedirs(checkpoint_dir, exist_ok=True)
checkpoint_callback = ModelCheckpoint(
    filepath=os.path.join(checkpoint_dir, 'model_epoch_{epoch:02d}.keras'),
    save_weights_only=False,
    save_freq='epoch'
)

history = model_with_masked_output.fit(
    train_generator,
    epochs=epochs,
    validation_data=val_generator,
    callbacks=[checkpoint_callback]
)


INFO:albumentations.check_version:A new version of Albumentations is available: 1.4.14 (you have 1.4.13). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.


Found 1208 files belonging to 24 classes.
Found 1208 files belonging to 24 classes.
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 270ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 268ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 278ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 269ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 274ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 269ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 283ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 275ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 266ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 275ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 285ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 274ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 275ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 274ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 272ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 271ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 270ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 275ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step  
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 277ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 282ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 276ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 300ms/step
2/2 ━━━━━━━━━━━━━━━━

Epoch 1/300


i:\anaconda\envs\tf-gpu\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


89/89 ━━━━━━━━━━━━━━━━━━━━ 76s 764ms/step - Precision: 0.0997 - Recall: 6.3769e-04 - accuracy: 0.0915 - categorical_accuracy: 0.0915 - loss: 3.3752 - val_Precision: 0.6667 - val_Recall: 0.0083 - val_accuracy: 0.2194 - val_categorical_accuracy: 0.2194 - val_loss: 2.6279
Epoch 2/300
89/89 ━━━━━━━━━━━━━━━━━━━━ 67s 735ms/step - Precision: 0.7940 - Recall: 0.0109 - accuracy: 0.2530 - categorical_accuracy: 0.2530 - loss: 2.5722 - val_Precision: 0.8462 - val_Recall: 0.0546 - val_accuracy: 0.4106 - val_categorical_accuracy: 0.4106 - val_loss: 2.0306
Epoch 3/300
89/89 ━━━━━━━━━━━━━━━━━━━━ 66s 724ms/step - Precision: 0.9229 - Recall: 0.0654 - accuracy: 0.4139 - categorical_accuracy: 0.4139 - loss: 2.1179 - val_Precision: 0.9108 - val_Recall: 0.1606 - val_accuracy: 0.5530 - val_categorical_accuracy: 0.5530 - val_loss: 1.6467
Epoch 4/300
89/89 ━━━━━━━━━━━━━━━━━━━━ 67s 733ms/step - Precision: 0.9525 - Recall: 0.1533 - accuracy: 0.5128 - categorical_accuracy: 0.5128 - loss: 1.8385 - val_Precision: 0

In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD
import albumentations as A
import os
import random
from tensorflow.keras.callbacks import ModelCheckpoint


train_dir = 'O:/project/dataset/train'
val_dir = 'O:/project/dataset/val'
checkpoint_dir = 'O:/project/epochs/mobile_adam_efined50_newmodel_freezed'
initial_learning_rate = 1e-4
epochs = 300

# Parameters
batch_size = 64
img_height = 160
img_width = 160
num_classes = 24  # Adjust this based on your specific task
val_dir = 'O:/project/dataset/val'
adv_val_dir = 'O:/project/dataset/fsgm_mobile_adam_fined_e50/epsilon_0.1/val'
model_path = 'O:/project/epochs/check_mobilenetv2_adam_4_e50_fined_50/model_epoch_50.keras'

# Preprocessing function to rescale images
def preprocess(image, label):
    image = tf.cast(image, tf.float32) / 255.0  # Rescale image to [0, 1]
    return image, label

# 1. Load the Images from Directories with preprocessing
test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    val_dir,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    label_mode='categorical'  # Assuming labels are categorical
).map(preprocess)

adv_test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    adv_val_dir,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    label_mode='categorical'
).map(preprocess)

# 2. Load the pre-trained model
model = tf.keras.models.load_model(model_path)

# 3. Remove the last layers (classifier)
layer_name = model.layers[-2].name  # Get the name of the layer before the final classifier layer
model_without_classifier = models.Model(inputs=model.input, outputs=model.get_layer(layer_name).output)

# 4. Extract Features for Test and Adversarial Test datasets
def extract_features(model, dataset):
    features = []
    labels = []
    for images, lbls in dataset:
        feats = model.predict(images)
        features.append(feats)
        labels.append(lbls)
    return np.concatenate(features), np.concatenate(labels)

test_features, test_labels = extract_features(model_without_classifier, test_dataset)
adv_test_features, _ = extract_features(model_without_classifier, adv_test_dataset)

# 5. Compare the Outputs (identify significant node changes)
node_changes = np.abs(test_features - adv_test_features).mean(axis=0)

# Determine a threshold to ignore nodes (you can define it based on your criteria)
threshold = np.mean(node_changes) + np.std(node_changes)  # Example threshold
ignored_nodes = node_changes > threshold

print(f"Threshold: {threshold}")
a = np.sum(ignored_nodes)
print(f"Number of ignored nodes: {a}")




np.save(checkpoint_dir +'ignored_nodes.npy', ignored_nodes)


Found 1208 files belonging to 24 classes.
Found 1208 files belonging to 24 classes.
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 341ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 281ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 301ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 283ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 285ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 277ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 349ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 308ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 281ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 293ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 274ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 265ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 281ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 273ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 281ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 274ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 276ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 332ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step  
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 275ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 329ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 299ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 298ms/step
2/2 ━━━━━━━━━━━━━━━━